# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f899cc58580>
├── 'a' --> tensor([[-1.0312,  1.5261,  0.6664],
│                   [-0.0090, -0.1369,  0.9733]])
└── 'x' --> <FastTreeValue 0x7f899ccb8fd0>
    └── 'c' --> tensor([[-0.4424,  0.8297, -1.0981, -0.1495],
                        [-0.6674, -1.0596,  1.2626, -1.4390],
                        [-0.8467,  0.1597,  0.4012,  0.7591]])

In [4]:
t.a

tensor([[-1.0312,  1.5261,  0.6664],
        [-0.0090, -0.1369,  0.9733]])

In [5]:
%timeit t.a

65.8 ns ± 0.0258 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f899cc58580>
├── 'a' --> tensor([[ 0.5699, -0.9392, -1.3374],
│                   [ 0.2876,  0.3043, -1.2834]])
└── 'x' --> <FastTreeValue 0x7f899ccb8fd0>
    └── 'c' --> tensor([[-0.4424,  0.8297, -1.0981, -0.1495],
                        [-0.6674, -1.0596,  1.2626, -1.4390],
                        [-0.8467,  0.1597,  0.4012,  0.7591]])

In [7]:
%timeit t.a = new_value

62.1 ns ± 0.0338 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.0312,  1.5261,  0.6664],
               [-0.0090, -0.1369,  0.9733]]),
    x: Batch(
           c: tensor([[-0.4424,  0.8297, -1.0981, -0.1495],
                      [-0.6674, -1.0596,  1.2626, -1.4390],
                      [-0.8467,  0.1597,  0.4012,  0.7591]]),
       ),
)

In [10]:
b.a

tensor([[-1.0312,  1.5261,  0.6664],
        [-0.0090, -0.1369,  0.9733]])

In [11]:
%timeit b.a

56.4 ns ± 0.0392 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.4604,  0.1842, -0.3352],
               [ 2.5018,  0.7386, -1.1681]]),
    x: Batch(
           c: tensor([[-0.4424,  0.8297, -1.0981, -0.1495],
                      [-0.6674, -1.0596,  1.2626, -1.4390],
                      [-0.8467,  0.1597,  0.4012,  0.7591]]),
       ),
)

In [13]:
%timeit b.a = new_value

524 ns ± 0.873 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

842 ns ± 0.171 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 24.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 286 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 468 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f88e6f6c790>
├── 'a' --> tensor([[[-1.0312,  1.5261,  0.6664],
│                    [-0.0090, -0.1369,  0.9733]],
│           
│                   [[-1.0312,  1.5261,  0.6664],
│                    [-0.0090, -0.1369,  0.9733]],
│           
│                   [[-1.0312,  1.5261,  0.6664],
│                    [-0.0090, -0.1369,  0.9733]],
│           
│                   [[-1.0312,  1.5261,  0.6664],
│                    [-0.0090, -0.1369,  0.9733]],
│           
│                   [[-1.0312,  1.5261,  0.6664],
│                    [-0.0090, -0.1369,  0.9733]],
│           
│                   [[-1.0312,  1.5261,  0.6664],
│                    [-0.0090, -0.1369,  0.9733]],
│           
│                   [[-1.0312,  1.5261,  0.6664],
│                    [-0.0090, -0.1369,  0.9733]],
│           
│                   [[-1.0312,  1.5261,  0.6664],
│                    [-0.0090, -0.1369,  0.9733]]])
└── 'x' --> <FastTreeValue 0x7f88e6f6c580>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.2 µs ± 63 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f88e779ff70>
├── 'a' --> tensor([[-1.0312,  1.5261,  0.6664],
│                   [-0.0090, -0.1369,  0.9733],
│                   [-1.0312,  1.5261,  0.6664],
│                   [-0.0090, -0.1369,  0.9733],
│                   [-1.0312,  1.5261,  0.6664],
│                   [-0.0090, -0.1369,  0.9733],
│                   [-1.0312,  1.5261,  0.6664],
│                   [-0.0090, -0.1369,  0.9733],
│                   [-1.0312,  1.5261,  0.6664],
│                   [-0.0090, -0.1369,  0.9733],
│                   [-1.0312,  1.5261,  0.6664],
│                   [-0.0090, -0.1369,  0.9733],
│                   [-1.0312,  1.5261,  0.6664],
│                   [-0.0090, -0.1369,  0.9733],
│                   [-1.0312,  1.5261,  0.6664],
│                   [-0.0090, -0.1369,  0.9733]])
└── 'x' --> <FastTreeValue 0x7f88e7543b80>
    └── 'c' --> tensor([[-0.4424,  0.8297, -1.0981, -0.1495],
                        [-0.6674, -1.0596,  1.2626, -1.4390],
                 

In [23]:
%timeit t_cat(trees)

29.5 µs ± 56.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.8 µs ± 606 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.4424,  0.8297, -1.0981, -0.1495],
                       [-0.6674, -1.0596,  1.2626, -1.4390],
                       [-0.8467,  0.1597,  0.4012,  0.7591]],
              
                      [[-0.4424,  0.8297, -1.0981, -0.1495],
                       [-0.6674, -1.0596,  1.2626, -1.4390],
                       [-0.8467,  0.1597,  0.4012,  0.7591]],
              
                      [[-0.4424,  0.8297, -1.0981, -0.1495],
                       [-0.6674, -1.0596,  1.2626, -1.4390],
                       [-0.8467,  0.1597,  0.4012,  0.7591]],
              
                      [[-0.4424,  0.8297, -1.0981, -0.1495],
                       [-0.6674, -1.0596,  1.2626, -1.4390],
                       [-0.8467,  0.1597,  0.4012,  0.7591]],
              
                      [[-0.4424,  0.8297, -1.0981, -0.1495],
                       [-0.6674, -1.0596,  1.2626, -1.4390],
                       [-0.8467,  0.1597,  0.4012,  0.7591]],

In [26]:
%timeit Batch.stack(batches)

82.7 µs ± 286 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.4424,  0.8297, -1.0981, -0.1495],
                      [-0.6674, -1.0596,  1.2626, -1.4390],
                      [-0.8467,  0.1597,  0.4012,  0.7591],
                      [-0.4424,  0.8297, -1.0981, -0.1495],
                      [-0.6674, -1.0596,  1.2626, -1.4390],
                      [-0.8467,  0.1597,  0.4012,  0.7591],
                      [-0.4424,  0.8297, -1.0981, -0.1495],
                      [-0.6674, -1.0596,  1.2626, -1.4390],
                      [-0.8467,  0.1597,  0.4012,  0.7591],
                      [-0.4424,  0.8297, -1.0981, -0.1495],
                      [-0.6674, -1.0596,  1.2626, -1.4390],
                      [-0.8467,  0.1597,  0.4012,  0.7591],
                      [-0.4424,  0.8297, -1.0981, -0.1495],
                      [-0.6674, -1.0596,  1.2626, -1.4390],
                      [-0.8467,  0.1597,  0.4012,  0.7591],
                      [-0.4424,  0.8297, -1.0981, -0.1495],
                   

In [28]:
%timeit Batch.cat(batches)

148 µs ± 182 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

308 µs ± 1.97 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
